In [7]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper

    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses

    scraping.contract_addresses.main(local=True)

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.
    import gbq.gbq

    gbq.gbq.main(local=True)


main()


2023-04-04 11:25:40,846 - INFO - Cloud function starting.
2023-04-04 11:25:43,083 - INFO - Scraping complete.
2023-04-04 11:25:43,741 - INFO - Current working directory: /Users/dbless/Documents/Github/cryptocurrency-notifications/cloud_functions/coingecko_gainers_traders
2023-04-04 11:25:45,514 - INFO - Scraping 3 coins that are missing symbols.
2023-04-04 11:25:50,639 - INFO - Scraped https://www.coingecko.com/en/coins/crypto-gladiator-league
2023-04-04 11:25:54,330 - INFO - Scraped https://www.coingecko.com/en/coins/hnc-coin
2023-04-04 11:25:58,004 - INFO - Scraped https://www.coingecko.com/en/coins/jenny-dao-v1
2023-04-04 11:25:58,021 - INFO - Number of coins with ethereum contract addresses: 33
2023-04-04 11:25:58,022 - INFO - Done scraping contract addresses!
2023-04-04 11:25:58,027 - INFO - Checking 33 token contracts.
2023-04-04 11:25:58,285 - INFO - Running SQL query on Google Big Cloud.
2023-04-04 11:26:01,379 - INFO - Query will process 764 MB.
2023-04-04 11:26:01,380 - INFO 

In [10]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
import google.auth.transport.requests
import google.oauth2.id_token
import json

load_dotenv()
KEY_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
GOOGLE_CLOUD_FUNCTION = os.getenv("GOOGLE_CLOUD_FUNCTION")
endpoint = GOOGLE_CLOUD_FUNCTION
audience = endpoint

auth_req = google.auth.transport.requests.Request()
id_token = google.oauth2.id_token.fetch_id_token(auth_req, audience)

# Add message in request args
headers = {"Authorization": f"Bearer {id_token}", "Content-Type": "application/json"}


r = requests.post(
    endpoint, data=json.dumps({"message": "wow json data"}), headers=headers
)


In [13]:
import pandas as pd


<Response [200]>